In [16]:
with open("austen.txt", 'r', encoding='utf-8') as f1:
    text = f1.read()

In [13]:
print(f"length of the dataset in characters: {len(text)}")

length of the dataset in characters: 5292297


In [18]:
print(text[:1000])

No one who had ever seen Catherine Morland in her infancy would have supposed her born to be an heroine. Her situation in life, the character of her father and mother, her own person and disposition, were all equally against her. Her father was a clergyman, without being neglected, or poor, and a very respectable man, though his name was Richard--and he had never been handsome. He had a considerable independence besides two good livings--and he was not in the least addicted to locking up his daughters. Her mother was a woman of useful plain sense, with a good temper, and, what is more remarkable, with a good constitution. She had three sons before Catherine was born; and instead of dying in bringing the latter into the world, as anybody might expect, she still lived on--lived to have six children more--to see them growing up around her, and to enjoy excellent health herself. A family of ten children will be always called a fine family, where there are heads and arms and legs enough for

In [20]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("unique characters in the text: ", "".join(chars))
print("length of the vocabulary: ", vocab_size)

unique characters in the text:  
 !"&'()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^_abcdefghijklmnopqrstuvwxyz{}£½àáæèéê“”
length of the vocabulary:  94


In [22]:
# create a mapping from string to integers
stoi = {chars[i]:i for i in range(vocab_size)}
itos = {i:chars[i] for i in range(vocab_size)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda i: "".join([itos[j] for j in i])

print(encode("hii there"))
print(decode(encode("hii there")))

[63, 64, 64, 1, 75, 63, 60, 73, 60]
hii there


In [1]:
not set()

True

In [11]:
try:
    assert 
    assert not False
    print("Test Passed!")
except AssertionError:
    print("Test Failed")
    raise

Test Failed


AssertionError: 